In [1]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI,OpenAIEmbeddings #this to above classes we used to interact with llm model.
from dotenv import load_dotenv
from langgraph.graph import StateGraph,END,START #this class we used to build stateful workflows in graphical form mei.
from pydantic import BaseModel,Field,computed_field
from typing import Annotated,List,Dict,TypedDict,Optional
from langchain_core.output_parsers import PydanticOutputParser,StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint

load_dotenv()

True

# step1) define the LLM model object

In [2]:
#first we have to define kon se HuggingFaceEndpoint pe API Request jayeghi
# Hugging Face endpoint define karo
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
)

# Chat model object banao
model1 = ChatHuggingFace(llm=llm)
model2 = ChatOpenAI(temperature=0.2)

# defining structure output by using pydantic class this will return struture result from LLM

In [3]:
from typing import Literal
#structure schema defining to get response from LLM in this Way.
class EssayFeedbackScore(BaseModel):
    feedback: Annotated[
        Literal[
            "Clarity of thought",
            "Depth of analysis",
            "Language quality"
        ],
        str
    ] = Field(
        ...,
        description="Category of essay feedback: clarity of thought, depth of analysis, or language quality."
    )

    score: Annotated[
        int,
        Field(..., description="Score for the essay on a scale of 1 to 10 based on the selected feedback category.",ge=0,le=10)
    ]
    
    explanation: Annotated[
        str,
        Field(..., description="A Detailed Feedback For The Essay.")
    ]

In [4]:
#creating an object of pydantic output parser class
parser  = StrOutputParser()
pydantic_parser = PydanticOutputParser(pydantic_object=EssayFeedbackScore)
pydantic_parser

PydanticOutputParser(pydantic_object=<class '__main__.EssayFeedbackScore'>)

# testing pydantic parser

In [5]:
prompt = PromptTemplate(
    template="""
    You are an expert evaluator of essays. 
    Read the essay below and evaluate it based on **one** of the following categories only:
    - Clarity of thought
    - Depth of analysis
    - Language quality

    Then:
    - Select the most relevant feedback category.
    - Assign a score between 0 and 10.
    - Provide a short detailed explanation justifying the score.

    Essay:
    {essay}

    Return the result strictly following this format:
    {format_instructions}
    """,
    input_variables=['essay'],
    partial_variables={'format_instructions': pydantic_parser.get_format_instructions()}
)


with open('essay.txt','r') as file:
    text = file.read()


#want to see the prompt.
prompt.invoke({'essay':text})

StringPromptValue(text='\n    You are an expert evaluator of essays. \n    Read the essay below and evaluate it based on **one** of the following categories only:\n    - Clarity of thought\n    - Depth of analysis\n    - Language quality\n\n    Then:\n    - Select the most relevant feedback category.\n    - Assign a score between 0 and 10.\n    - Provide a short detailed explanation justifying the score.\n\n    Essay:\n    Artificial Intelligence (AI) is transforming the world, and India is embracing this technology to solve real challenges. In healthcare, AI is helping doctors with faster diagnosis and better treatment decisions. Artificial intelligence is very important in world today.\nIt help many peoples to finish work fast.\nStudent also use AI for write homework and essay.\nSometime teacher not happy because student not learning.\nCompany like to use AI because machine not take rest.\nBut many worker afraid they job going lost.\nAI also make mistake because it not have human min

In [6]:
#passing this prompt to model
# chain = prompt | model1 | pydantic_parser

# response = chain.invoke({'essay':text})
# response

#response.explanation


# step:1) defining the state or Memory Field using TypedDict or pydantic this state will be sharing throught workflow then creating graph object

In [7]:
import operator
class UpscEssayState(BaseModel):
    essay_txt: str = Field(..., description="Essay text provided by user")
    
    COT_feedback: Optional[str] = Field(None, description="Feedback for Clarity of Thought")
    DOA_feedback: Optional[str] = Field(None, description="Feedback for Depth of Analysis")
    LQ_feedback: Optional[str] = Field(None, description="Feedback for Language Quality")
    
    overall_feedback: Optional[str] = Field(None, description="Final summarized feedback")
    
    # Reducer: collect all node scores in a list
    individual_score_each_nodes: Annotated[
        List[int], operator.add
    ] = Field(default_factory=list, description="Scores returned by multiple nodes, combined via reducer")
    
    final_score: Optional[float] = Field(None, ge=0, le=10, description="Final averaged score")

In [8]:
# creating a graph object by using stategraph class.
graph = StateGraph(UpscEssayState)
graph

# step:2) now adding nodes or edges to the graph after then starting workflow

In [9]:
#creating evaluate_thought nodes ke liye user defined function and performing action init.
def evaluate_thought(state:UpscEssayState) ->UpscEssayState:
    
    text_essay = state.essay_txt
    
    #Now this essay we change them to structured instruction Manner.
    prompt = PromptTemplate(
    template="""
    You are an expert evaluator of essays. 
    Read the essay below and evaluate it strictly on **Clarity of Thought**.

    Then:
    - Assign a score between 0 and 10.
    - Provide a short detailed explanation justifying the score.

    Essay:
    {essay}

    Return the result strictly following this format:
    {format_instructions}
    """,
    input_variables=['essay'],
    partial_variables={'format_instructions': pydantic_parser.get_format_instructions()}
)
    
    
    #now passing this prompt to the structure response from LLM using chains
    chain = prompt | model2 | pydantic_parser
    
    #invoking the or calling the chain to get response from llm.
    result = chain.invoke({'essay':text_essay})
    
    #now updating the state with partial update and return back the updated state.
    
    return {
            "COT_feedback":result.explanation, # explanation from LLM
            "individual_score_each_nodes": [result.score]
            }

In [10]:
def evaluate_analysis(state:UpscEssayState) ->UpscEssayState:
    text_essay = state.essay_txt
    
    #Now this essay we change them to structured instruction Manner.
    prompt = PromptTemplate(
    template="""
    You are an expert evaluator of essays. 
    Read the essay below and evaluate it strictly on **Depth of Analysis**.

   Then:
    - Assign a score between 0 and 10.
    - Provide a short detailed explanation justifying the score.
    
    Essay:
    {essay}

    Return the result strictly following this format:
    {format_instructions}
    """,
    input_variables=['essay'],
    partial_variables={'format_instructions': pydantic_parser.get_format_instructions()}
)
    
    
    #now passing this prompt to the structure response from LLM using chains
    chain = prompt | model2 | pydantic_parser
    
    #invoking the or calling the chain to get response from llm.
    result = chain.invoke({'essay':text_essay})
    
    #now updating the state with partial update and return back the updated state.
    
    return {
            "DOA_feedback":result.explanation, # explanation from LLM
            "individual_score_each_nodes": [result.score]
            }

In [11]:
def evaluate_language(state:UpscEssayState) ->UpscEssayState:
    text_essay = state.essay_txt
    
    #Now this essay we change them to structured instruction Manner.
    prompt = PromptTemplate(
    template="""
    You are an expert evaluator of essays. 
    Read the essay below and evaluate it strictly on **Language Quality**.
    
    Then:
    - Assign a score between 0 and 10.
    - Provide a short detailed explanation justifying the score.
    
    Essay:
    {essay}

    Return the result strictly following this format:
    {format_instructions}
    """,
    input_variables=['essay'],
    partial_variables={'format_instructions': pydantic_parser.get_format_instructions()}
)
    
    
    #now passing this prompt to the structure response from LLM using chains
    chain = prompt | model2 | pydantic_parser
    
    #invoking the or calling the chain to get response from llm.
    result = chain.invoke({'essay':text_essay})
    
    #now updating the state with partial update and return back the updated state.
    
    return {
            "LQ_feedback":result.explanation, # explanation from LLM
            "individual_score_each_nodes": [result.score]
            }
    

In [12]:
def evaluate_final_essay(state:UpscEssayState) ->UpscEssayState:
    cot = state.COT_feedback
    doa = state.DOA_feedback
    lq = state.LQ_feedback
    individual_score = state.individual_score_each_nodes
    
    #calculating average score.
    avg_score = sum(individual_score)/len(individual_score)
    
    #creating structure instruction prompt.
    final_prompt = PromptTemplate(
        template="""
        You are an expert evaluator of essays. 
        Summarize the following feedback categories into a final review:
        - Clarity of thought: {cot}
        - Depth of analysis: {doa}
        - Language quality: {lq}

        Also consider the average score {scores:.2f} as the final score.
        
        Provide a well-structured overall feedback paragraph.
        """,
        input_variables=["cot", "doa", "lq", "scores"]
)
    
    #passing this prompt to LLM model forming chain.
    chain = final_prompt | model2 | parser
    

    
    #invoking the chain.
    response_llm = chain.invoke({"cot":cot, "doa":doa, "lq":lq, "scores":avg_score})
  
    
    #updating the state and returning the updated state.
    return {
        
        'overall_feedback':response_llm,
        "final_score": avg_score
    }
    

In [13]:
#adding nodes to graph.
#nodes is nothing but python function in which we r defing actions to perform.
graph.add_node(node="evaluate_thought",action=evaluate_thought)
graph.add_node(node="evaluate_analysis",action=evaluate_analysis)
graph.add_node(node="evaluate_language",action=evaluate_language)
graph.add_node(node="final_evaluate_essay",action=evaluate_final_essay)

In [14]:
#now adding the edges to graph edges will connect the nodes whenever reqd we can set loop or condition to edge.
graph.add_edge(START,'evaluate_thought')
graph.add_edge(START,'evaluate_analysis')
graph.add_edge(START,'evaluate_language')

graph.add_edge('evaluate_thought','final_evaluate_essay')
graph.add_edge('evaluate_analysis','final_evaluate_essay')
graph.add_edge('evaluate_language','final_evaluate_essay')

graph.add_edge('final_evaluate_essay',END)



# step:3) compiling the graph

In [15]:
#graph.compile()

# step:4) evaluating the graph

In [16]:
workflow = graph.compile()

#passing this initial state to graph or starting point.
intitial_state = UpscEssayState(
    essay_txt=text
)
intitial_state

UpscEssayState(essay_txt='Artificial Intelligence (AI) is transforming the world, and India is embracing this technology to solve real challenges. In healthcare, AI is helping doctors with faster diagnosis and better treatment decisions. Artificial intelligence is very important in world today.\nIt help many peoples to finish work fast.\nStudent also use AI for write homework and essay.\nSometime teacher not happy because student not learning.\nCompany like to use AI because machine not take rest.\nBut many worker afraid they job going lost.\nAI also make mistake because it not have human mind.\nSome people say AI is very danger for future.\nOther people think it is only tool for help.\nSo AI is good and bad both together. The future of AI in India is both promising and impactful.', COT_feedback=None, DOA_feedback=None, LQ_feedback=None, overall_feedback=None, individual_score_each_nodes=[], final_score=None)

# passing this initial to my parallel workflow

In [17]:
result = workflow.invoke(intitial_state)
result

{'essay_txt': 'Artificial Intelligence (AI) is transforming the world, and India is embracing this technology to solve real challenges. In healthcare, AI is helping doctors with faster diagnosis and better treatment decisions. Artificial intelligence is very important in world today.\nIt help many peoples to finish work fast.\nStudent also use AI for write homework and essay.\nSometime teacher not happy because student not learning.\nCompany like to use AI because machine not take rest.\nBut many worker afraid they job going lost.\nAI also make mistake because it not have human mind.\nSome people say AI is very danger for future.\nOther people think it is only tool for help.\nSo AI is good and bad both together. The future of AI in India is both promising and impactful.',
 'COT_feedback': "The essay lacks clarity of thought as it jumps between different points without proper transitions or development. The ideas presented are fragmented and do not flow logically. The use of examples is